In [1]:
import os
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

dir_maps = './'
dir_logcube = './'




In [6]:
hdu[1].data['plate'][0]

10001

In [4]:
hdu = fits.open('../SDSS17Pipe3D_v3_1_1_part.fits')
hdu[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional table                            
NAXIS1  =                 3155 / width of table in bytes                        
NAXIS2  =                10220 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group                                 
TFIELDS =                  392 / number of columns                              
EXTNAME = 'GAL_PROPERTIES'     / table name                                     
TTYPE1  = 'name    '           / label for column 1                             
TFORM1  = '32A     '           / format for column 1                            
TTYPE2  = 'plate   '           / label for column 2                             
TFORM2  = 'K       '        

In [5]:
len(fits.open('../dapall-v3_1_1-3.1.0.fits')[1].data['PLATE'])

10782

In [2]:
hdu = fits.open('../dapall-v3_1_1-3.1.0.fits')

In [9]:
hdu[1].data['IFUDESIGN']

array([ 1902, 12704,  1901, ...,  3701,  1901, 12705], dtype='>i8')

In [ ]:
def stack_logcube(region, cubefile=None, mapsfile=None,
                  rmin=0., rmax=-1., pamin=0., pamax=360., recustom=-1.0,):
    """Stack DAP LOGCUBE spectra based on given morphology mode.

    Args:
        mode (str): 'online' or 'local'.
        region (str): Type of stacked region. Supported: 'all', 'cen', 'rering', 'pasector'.
        plate (int, optional): Plate number, only for 'online' mode. Defaults to None.
        ifudesign (int, optional): IFU number, only for 'online' mode. Defaults to None.
        cubefile (str, optional): File path for LOGCUBE file, only for 'local' mode. Defaults to None.
        mapsfile (str, optional): File path for MAPS file, only for 'local' mode. Defaults to None.
        rmin (float, optional): For 'rering' or 'pasector' mode, the inner radius of the ring or sector region. Defaults to 0.0.
        rmax (float, optional): For 'rering' or 'pasector' mode, the outer radius of the ring region. Will be changed to resolution if remax<fwhm. Defaults to -1.0.
        pamin (float, optional): For 'pasector' mode, the start PA in clockwise direction. Defaults to 0.0.
        pamax (float, optional): For 'pasector' mode, the end PA in clockwise direction. Defaults to 360.0.
        recustom (float, optional): For 'rering' mode, the 1 unit radius in arcsec. Must be positive number.
        
    Returns:
        turple with 4: observed wavelength, stacked spectra, 1 sigma error, number of spectra used for stacking along the wavelength
    """    
    
    if ~(np.sum(region == np.array(['all', 'cen', 'rering', 'pasector'])) == 1):
        raise RuntimeError("Stack region should be 'tot', 'cen', 'rering', or 'pasector'!")
    
    hducube = fits.open(cubefile)
    hdumaps = fits.open(mapsfile)

    try:  
        wave = hducube['WAVE'].data
        flux = hducube['FLUX'].data
        ivar = hducube['IVAR'].data
        mask = hducube['MASK'].data
        remap = hdumaps['SPX_ELLCOO'].data[1, :, :]
        # kpcmap = hdumaps['SPX_ELLCOO'].data[2, :, :] / 0.7
        pamap = hdumaps['SPX_ELLCOO'].data[3, :, :]
        rfwhm = hdumaps[0].header['RFWHM'] / 2.0
    except Exception:
        raise RuntimeError("Not valid LOGCUBE or MAPS file!")

    try:
        bta = 1.0 - hdumaps[0].header['ECOOELL']
        # redap = hdumaps[0].header['REFF']
        padap = hdumaps[0].header['ECOOPA']
    except Exception:
        bta = 1.0
        # redap = 1.0
        padap = 0.0
        print("Warning: No valid 'Re' measurement!")

    arcmap = hdumaps['SPX_ELLCOO'].data[0, :, :]
    mapmaxarc, mapmaxre = np.nanmax(arcmap), np.nanmax(remap)
    rmax = rmax if rmax > 0 else np.max([mapmaxarc, mapmaxre])
    roundmap = np.sqrt((np.cos(pamap / 180.0 * np.pi) * arcmap)**2 + 
                       (np.sin(pamap / 180.0 * np.pi) * arcmap * bta)**2)

    if region == 'all':
        rcut = remap >= 0
    elif region == 'cen':
        rcut = roundmap <= 1.5  # rfwhm
    elif (region == 'rering'):
        if (rmax * mapmaxarc / mapmaxre) < rfwhm:
            print("Warning: 'rmax' is smaller than the resolution!")
        if recustom > 0:
            remap = arcmap / recustom
        rcut = (remap <= rmax) & (remap >= rmin)
    elif region == 'pasector':
        if (pamax > pamin):
            if (rmax * mapmaxarc / mapmaxre) < rfwhm:
                print("Warning: 'rmax' is smaller than the resolution! Return empty spectra.")
            pamap = np.mod(pamap + padap, 360.0) - pamin        
            rcut = (remap <= rmax) & (remap >= rmin) & (roundmap >= rfwhm) & ((pamap >= 0) & (pamap <= (pamax - pamin)) | (pamap >= 180.0) & (pamap <= (pamax - pamin + 180.0)))
        else:
            raise RuntimeError("Invalid 'pamin' or 'pamax'! Define them in clockwise direction.")
    
    nstack = np.sum(rcut)
    if nstack == 0:
        print("Warning: No spectra found in defined region! Return empty spectra.")
        return (wave, np.zeros_like(wave) - np.nan, np.zeros_like(wave) - np.nan, np.zeros_like(wave, dtype=int))
    else:
        flux[mask > 0] = np.nan  # mask 5578 skyline
        ivar[mask > 0] = np.nan
        goodspec = np.sum(~np.isnan(flux[:, rcut]), axis=1)
        goodspec[(wave >= 5570) & (wave <= 5586)] = 0
        fluxstack = np.nanmean(flux[:, rcut], axis=1) * goodspec
        errstack = np.sqrt(np.nanmean(1. / ivar[:, rcut], axis=1) * goodspec) 
        fluxstack[(wave >= 5570) & (wave <= 5586)] = np.nan
        errstack[(wave >= 5570) & (wave <= 5586)] = np.nan
        
        return wave, fluxstack, errstack, goodspec
    
    hducube.close()
    hdumaps.close()  

In [ ]:

file_maps = './manga-7443-12703-MAPS-SPX-MILESHC-MASTARSSP.fits.gz'
file_logcube = './manga-7443-12703-LOGCUBE-SPX-MILESHC-MASTARSSP.fits.gz'
# maps = fits.open(file_maps)
w_test, f_test, ef_test, gspec_test = stack_logcube('remap', )

In [5]:
maps.info()

Filename: ./manga-7443-12703-MAPS-SPX-MILESHC-MASTARSSP.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     141   ()      
  1  SPX_SKYCOO    1 ImageHDU        44   (74, 74, 2)   float32   
  2  SPX_ELLCOO    1 ImageHDU        49   (74, 74, 4)   float32   
  3  SPX_MFLUX     1 ImageHDU        37   (74, 74)   float32   
  4  SPX_MFLUX_IVAR    1 ImageHDU        38   (74, 74)   float32   
  5  SPX_SNR       1 ImageHDU        35   (74, 74)   float32   
  6  BINID         1 ImageHDU        46   (74, 74, 5)   int32   
  7  BIN_LWSKYCOO    1 ImageHDU        44   (74, 74, 2)   float32   
  8  BIN_LWELLCOO    1 ImageHDU        49   (74, 74, 4)   float32   
  9  BIN_AREA      1 ImageHDU        36   (74, 74)   float32   
 10  BIN_FAREA     1 ImageHDU        35   (74, 74)   float32   
 11  BIN_MFLUX     1 ImageHDU        38   (74, 74)   float32   
 12  BIN_MFLUX_IVAR    1 ImageHDU        39   (74, 74)   float32   
 13  BIN_MFLUX_MASK    1 Image